In [1]:
import os
import sys
import time
import json
from os.path import abspath, dirname
import pandas as pd
import numpy as np
import datetime

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngine")

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngineBase")

import synthesis
import coins.btcflow as btcflow

In [30]:
class btcSynth():

    def __init__(self, level_size, exchanges = ["binance", "okx", "bybit", "bitget", "bingx", "mexc", "kucoin", "deribit", "coinbase", "htx", "gateio"]):
        self.exchanges = exchanges
        self.level_size = level_size
        self.axis = {
            "binance" : btcflow.binance_flow(level_size),
            "okx" : btcflow.okx_flow(level_size),
            "bybit" : btcflow.bybit_flow(level_size),
            "bingx" : btcflow.bingx_flow(level_size),
            "mexc" : btcflow.mexc_flow(level_size),
            "kucoin" : btcflow.kucoin_flow(level_size),
            "deribit" : btcflow.deribit_flow(level_size),
            "coinbase" : btcflow.coinbase_flow(level_size),
            "htx" : btcflow.htx_flow(level_size),
            "gateio" : btcflow.gateio_flow(level_size),
            "bitget" : btcflow.bitget_flow(level_size),
        }
        for exchange in self.axis.keys():
            if exchange not in self.exchanges:
                del self.axis[exchange]

        self.books = {
            "spot" : synthesis.booksmerger("btc", "spot", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].spot_axis["books"][flow] for exchange in self.axis.keys() for flow in self.axis[exchange].spot_axis["books"].keys()
            }
                                           ),
            "perpetual" : synthesis.booksmerger("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["books"][flow] for exchange in self.axis.keys() for flow in self.axis[exchange].perpetual_axis["books"].keys()
            }
                                           ),
        }
        self.trades = {
            "spot" : synthesis.tradesmerger("btc", "spot", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].spot_axis["trades"][flow] for exchange in self.axis.keys() if exchange != "deribit" for flow in self.axis[exchange].spot_axis["trades"].keys()
            }
                                           ),
            "perpetual" : synthesis.tradesmerger("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["trades"][flow] for exchange in self.axis.keys() if exchange != "coinbase" for flow in self.axis[exchange].perpetual_axis["trades"].keys() 
            }
                                           ),
        }

        self.oifunding =  synthesis.oiomnifier("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["oifunding"][flow] for exchange in self.axis.keys() if exchange != "coinbase" for flow in self.axis[exchange].perpetual_axis["oifunding"].keys() 
            }
                                           )

        self.liquidations =  synthesis.lomnifier("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["liquidations"][flow] for exchange in self.axis.keys() if exchange in ["binance", "okx", "bybit"] for flow in self.axis[exchange].perpetual_axis["liquidations"].keys() 
            }
                                           )

        self.positionsTTA =  synthesis.indomnifier("btc", "perpetul", "TTA", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "TTA" in flow
            }
                                           )

        self.positionsTTP =  synthesis.indomnifier("btc", "perpetul", "TTP", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "TTP" in flow
            }
                                           )

        self.positionsGTA =  synthesis.indomnifier("btc", "perpetul", "GTA", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance", "okx", "bybit"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "GTA" in flow
            }
                                           )
        self.data = dict()


    def merge(self):
        for insType in self.books.keys():
            try:
                self.books[insType].merge_snapshots()
                self.data["_".join(["books", insType])] = self.books[insType].snapshotO
            except:
                pass
        for insType in self.trades.keys():
            try:
                self.trades[insType].merge_snapshots()
                self.data["_".join(["trades", insType])] = self.trades[insType].snapshotO
            except:
                pass
        try:
            self.oifunding.merge_snapshots()
            self.data["_".join(["oifunding", "perpetual"])] = self.oifunding.snapshot
        except:
            pass
        try:
            self.liquidations.merge_snapshots()
            self.data["_".join(["liquidations", "perpetual"])] = self.liquidations.snapshot
        except:
            pass
        try:
            self.positionsTTA.merge_snapshots()
            self.data["_".join(["TTA", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        try:
            self.positionsTTP.merge_snapshots()
            self.data["_".join(["TTP", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        try:
            self.positionsGTA.merge_snapshots()
            self.data["_".join(["GTA", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        
    # Merge
    def ratrive_data(self, data=None, type_=None):
        """
            books_spot, books_perp, trades_spot, trades_perp, oifunding, liquidations, TTA, TTP, GTA
        """
        if data == None and type_ == None:
            return self.data
    
    # add data

    # Spot books

    def add_binance_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["binance_btcusdt"].update_books(data)

    def add_binance_spot_btcfdusd_depth(self, data):
        self.books["spot"].axis["binance_btcfdusd"].update_books(data)

    def add_okx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["okx_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bybit_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bybit_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdc_depth(self, data):
        self.books["spot"].axis["bybit_btcusdc"].update_books(data)

    def add_bingx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bingx_btcusdt"].update_books(data)

    def add_mexc_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["mexc_btcusdt"].update_books(data)

    def add_kucoin_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["kucoin_btcusdt"].update_books(data)

    def add_coinbase_spot_btcusd_depth(self, data):
        self.books["spot"].axis["coinbase_btcusd"].update_books(data)

    def add_htx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["htx_btcusdt"].update_books(data)

    def add_gateio_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["gateio_btcusdt"].update_books(data)

    def add_bitget_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bitget_btcusdt"].update_books(data)

    # Spot trades

    def add_binance_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["binance_btcusdt"].input_trades(data)

    def add_binance_spot_btcfdusd_trades(self, data):
        self.trades["spot"].axis["binance_btcfdusd"].input_trades(data)

    def add_okx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["okx_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdc_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdc"].input_trades(data)

    def add_bingx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bingx_btcusdt"].input_trades(data)

    def add_mexc_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["mexc_btcusdt"].input_trades(data)

    def add_kucoin_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["kucoin_btcusdt"].input_trades(data)

    def add_coinbase_spot_btcusd_trades(self, data):
        self.trades["spot"].axis["coinbase_btcusd"].input_trades(data)

    def add_htx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["htx_btcusdt"].input_trades(data)

    def add_gateio_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["gateio_btcusdt"].input_trades(data)

    def add_bitget_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bitget_btcusdt"].input_trades(data)

    # Perp books
    
    def add_binance_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["binance_btcusdt"].update_books(data) 

    def add_binance_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["binance_btcusd"].update_books(data) 

    def add_okx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["okx_btcusdt"].update_books(data) 

    def add_okx_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["okx_btcusd"].update_books(data)

    def add_bybit_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bybit_btcusdt"].update_books(data) 

    def add_bybit_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["bybit_btcusd"].update_books(data)

    def add_bingx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bingx_btcusdt"].update_books(data)   

    def add_mexc_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["mexc_btcusdt"].update_books(data) 

    def add_kucoin_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["kucoin_btcusdt"].update_books(data) 

    def add_deribit_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["deribit_btcusd"].update_books(data) 

    def add_htx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["htx_btcusdt"].update_books(data) 

    def add_gateio_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["gateio_btcusdt"].update_books(data) 

    def add_bitget_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bitget_btcusdt"].update_books(data) 

    # Perp trades

    def add_binance_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["binance_btcusdt"].input_trades(data) 

    def add_binance_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["binance_btcusd"].input_trades(data) 

    def add_okx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["okx_btcusdt"].input_trades(data) 

    def add_okx_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["okx_btcusd"].input_trades(data)

    def add_bybit_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bybit_btcusdt"].input_trades(data) 

    def add_bybit_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["bybit_btcusd"].input_trades(data)

    def add_bingx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bingx_btcusdt"].input_trades(data)   

    def add_mexc_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["mexc_btcusdt"].input_trades(data) 

    def add_kucoin_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["kucoin_btcusdt"].input_trades(data) 

    def add_deribit_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["deribit_btcusd"].input_trades(data) 

    def add_htx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["htx_btcusdt"].input_trades(data) 

    def add_gateio_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["gateio_btcusdt"].input_trades(data) 

    def add_bitget_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bitget_btcusdt"].input_trades(data)         

    # OI

    def add_binance_perp_btcusdt_oi(self, data):
        self.oifunding.axis["binance_btcusdt"].input_oi(data) 

    def add_binance_perp_btcusd_oi(self, data):
        self.oifunding.axis["binance_btcusd"].input_oi(data) 

    def add_okx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["okx_btcusdt"].input_oi(data) 

    def add_okx_perp_btcusd_oi(self, data):
        self.oifunding.axis["binance_btcusd"].input_oi(data) 

    def add_bingx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["bingx_btcusdt"].input_oi(data) 

    def add_gateio_perp_btcusdt_oi(self, data):
        self.oifunding.axis["gateio_btcusdt"].input_oi(data) 

    def add_htx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["htx_btcusdt"].input_oi(data) 
    
    # Funding

    def add_binance_perp_btcusdt_funding(self, data):
        self.oifunding.axis["binance_btcusdt"].input_funding(data) 

    def add_binance_perp_btcusd_funding(self, data):
        self.oifunding.axis["binance_btcusd"].input_funding(data) 

    def add_okx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["okx_btcusdt"].input_funding(data) 

    def add_okx_perp_btcusd_funding(self, data):
        self.oifunding.axis["okx_btcusd"].input_funding(data) 

    def add_bingx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["bingx_btcusdt"].input_funding(data) 

    def add_gateio_perp_btcusdt_funding(self, data):
        self.oifunding.axis["gateio_btcusdt"].input_funding(data) 

    def add_htx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["htx_btcusdt"].input_funding(data) 
    
    # OIFund  

    def add_bybit_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["bybit_btcusdt"].input_funding(data) 

    def add_bybit_perp_btcusd_oifunding(self, data):
        self.oifunding.axis["bybit_btcusd"].input_funding(data) 

    def add_mexc_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["mexc_btcusdt"].input_funding(data) 

    def add_kucoin_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["kucoin_btcusd"].input_funding(data) 

    def add_bitget_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["bitget_btcusdt"].input_funding(data) 

    def add_gateio_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["gateio_btcusdt"].input_funding(data) 

    def add_htx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["htx_btcusdt"].input_funding(data) 

    def add_deribit_perp_btcusd_oifunding(self, data):
        self.oifunding.axis["deribit_btcusd"].input_funding(data) 
    
    # Liquidations
    
    def add_binance_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["binance_btcusdt"].input_liquidations(data) 

    def add_binance_perp_btcusd_liquidations(self, data):
        self.liquidations.axis["binance_btcusd"].input_liquidations(data) 

    def add_okx_perp_btc_liquidations(self, data):
        self.liquidations.axis["okx_btc"].input_liquidations(data) 

    def add_bybit_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["bybit_btcusdt"].input_liquidations(data) 

    def add_bybit_perp_btcusd_liquidations(self, data):
        self.liquidations.axis["bybit_btcusd"].input_liquidations(data)

    def add_gateio_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["gateio_btcusdt"].input_liquidations(data)  
    
    # Positions

    def add_binance_perp_btcusdt_tta(self, data):
        self.positionsTTA.axis["binance_btcusdt_TTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_tta(self, data):
        self.positionsTTA.axis["binance_btcusd_TTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusdt_ttp(self, data):
        self.positionsTTP.axis["binance_btcusdt_TTP"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_ttp(self, data):
        self.positionsTTP.axis["binance_btcusd_TTP"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusdt_gta(self, data):
        self.positionsGTA.axis["binance_btcusdt_GTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_gta(self, data):
        self.positionsGTA.axis["binance_btcusd_GTA"].input_binance_gta_tta_ttp(data) 

    def add_okx_perp_btc_gta(self, data):
        self.positionsGTA.axis["okx_GTA"].input_okx_gta(data) 

    def add_bybit_perp_btcusdt_gta(self, data):
        self.positionsGTA.axis["bybit_btcusdt_GTA"].input_bybit_gta(data) 

    def add_bybit_perp_btcusd_gta(self, data):
        self.positionsGTA.axis["bybit_btcusd_GTA"].input_bybit_gta(data) 

    # Test Synth

    def input_from_json(self):
        """
            child directory data should contain json files
        """
        callable_methods = [method for method in dir(self) if callable(getattr(self, method)) and not method.startswith("__")]
        methods = []
        methods_str = []
        for method_name in callable_methods:
            if method_name not in ["ratrive_data", "merge", "input_from_json"]:
                method = getattr(self, method_name)
                methods.append(method)
                methods_str.append(method_name)    
        import os
        parent_dir = os.getcwd()
        data_dir  = parent_dir + '/data'
        files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
        methods_str = ["_".join(x.split("_")[1:]) + ".json" for x in sorted(methods_str)]
        files = [x for x in sorted(files) if x not in ["bybit_option_btc_oi.json", "okx_option_btc_oi.json", "deribit_option_btc_oi.json", "news.json"]]
        return list(set(files).difference(set(methods_str)))
        




In [31]:
a = btcSynth(20)
l = a.input_from_json()
l

['deribit_perp_btcusd_oifunding.json']

In [ ]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Specify the directory you want to change to
new_directory = "C:\coding\TradeStreamEngine"

# Change the current working directory
os.chdir(new_directory)

# Verify the change
updated_directory = os.getcwd()
print("Updated Working Directory:", updated_directory)

directory_path = "C:\coding\TradeStreamEngine\examples\data"

files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

print(files)



In [ ]:
files

In [ ]:
import inspect

callable_methods = [method for method in dir(a) if callable(getattr(a, method)) and not method.startswith("__")]
methods = []

for method_name in callable_methods:
    method = getattr(a, method_name)
    methods.append(method)

methods